In [29]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import pandas as pd
import re
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import unicodedata
from joblib import load
from transformers import CleanText, TokenizerText 


In [62]:
df = pd.read_csv('datos_productos.csv')

In [64]:
print("Valores nulos en 'Reviews':", df['Reviews'].isnull().sum())


Valores nulos en 'Reviews': 0


In [66]:
preprocessor = load('preprocessor.pkl')

In [68]:
preprocessor

ColumnTransformer(transformers=[('tipo', OneHotEncoder(drop='first'), ['Tipo']),
                                ('descripcion',
                                 Pipeline(steps=[('cleaner', CleanText()),
                                                 ('tokenizer', TokenizerText()),
                                                 ('tfidf',
                                                  TfidfVectorizer(vocabulary=['cabello',
                                                                              'piel',
                                                                              'facial',
                                                                              'natural',
                                                                              'tinte',
                                                                              'pelo',
                                                                              'corporal',
                                                                              'barba',
                                                                              'crecimiento',
                                                                              'cuerpo',
                                                                              'caida',
                                                                              'parabenos',
                                                                              'anticaida',
                                                                              'vegano',
                                                                              'canas',
                                                                              'cara',
                                                                              'fragancia',
                                                                              'organico',
                                                                              'afeitado',
                                                                              'aroma',
                                                                              'coloracion',
                                                                              'delicada',
                                                                              'ecologico',
                                                                              'exfoliante',
                                                                              'acne',
                                                                              'fortalecer',
                                                                              'graso',
                                                                              'hidrata',
                                                                              'poros',
                                                                              'tradicionales', ...]))]),
                                 'Product_Description'),
                                ('numeric',
                                 FunctionTransformer(func=<ufunc 'log1p'>),
                                 ['Price', 'Reviews'])])

In [70]:
# Separar el dataset en dos grupos según el número de reviews
df_menos_60 = df[df['Reviews'] < 60]   # Datos con menos de 60 reviews
df_mas_60 = df[df['Reviews'] >= 60]    # Datos con 60 o más reviews

# Verificar la distribución
print("Número de productos con <60 reviews:", len(df_menos_60))
print("Número de productos con >=60 reviews:", len(df_mas_60))

Número de productos con <60 reviews: 223
Número de productos con >=60 reviews: 172


In [107]:
from joblib import load

# Cargar el preprocesador previamente guardado
preprocessor = load('preprocessor.pkl')

def Featuring(df, preprocessor):
    # Verificar que las columnas necesarias existen
    required_columns = ['Tipo', 'Product_Description', 'Price', 'Reviews', 'Star_Rating']
    if not all(col in df.columns for col in required_columns):
        raise ValueError(f"El DataFrame debe contener las siguientes columnas: {required_columns}")
    
    # Separar el dataset en dos grupos
    df_menos_60 = df[df['Reviews'] < 60]   # Datos con menos de 60 reviews
    df_mas_60 = df[df['Reviews'] >= 60] 

    # Aplicar preprocesamiento a cada subconjunto
    X_bajo = df_menos_60[['Tipo', 'Product_Description', 'Price']]
    y_bajo = df_menos_60['Star_Rating']

    X_alto = df_mas_60[['Tipo', 'Product_Description', 'Price']]
    y_alto = df_mas_60['Star_Rating']

    # Preprocesar características
    X_bajo_preprocessed = preprocessor.transform(X_bajo)  # Usar el preprocesador cargado
    X_alto_preprocessed = preprocessor.transform(X_alto)  # Usar el preprocesador cargado
    
    return X_bajo_preprocessed, y_bajo, X_alto_preprocessed, y_alto

In [109]:
X_bajo_preprocessed, y_bajo, X_alto_preprocessed, y_alto = Featuring(df,preprocessor)

In [111]:
# Dividir y preprocesar los datos
X_bajo_preprocessed, y_bajo, X_alto_preprocessed, y_alto = Featuring(df, preprocessor)

# Entrenar modelos separados
model_bajo = RandomForestRegressor(random_state=42)
model_bajo.fit(X_bajo_preprocessed, y_bajo)

model_alto = RandomForestRegressor(random_state=42)
model_alto.fit(X_alto_preprocessed, y_alto)

NameError: name 'RandomForestRegressor' is not defined

In [186]:
# print(X_preprocessed[0:5])

In [113]:
# Concatenar X_preprocessed y y, asegurando que los índices coincidan
data_bajo = pd.concat([pd.DataFrame(X_bajo_preprocessed.toarray()), y_bajo.reset_index(drop=True)], axis=1)
data_alto = pd.concat([pd.DataFrame(X_alto_preprocessed.toarray()), y_alto.reset_index(drop=True)], axis=1)

In [123]:
data_bajo

,0,1,2,3,4,5,6,7,8,9,...,59,60,61,62,63,64,65,66,67,Star_Rating
0,0.0,0.0,0.365195,0.120966,0.000000,0.186483,0.0,0.347086,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.412866,2.771964,5.0
1,0.0,0.0,0.825656,0.097674,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,2.564180,4.4
2,0.0,0.0,0.648032,0.107326,0.000000,0.220608,0.0,0.000000,0.111123,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,2.557227,4.0
3,0.0,0.0,0.880050,0.104109,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,2.564180,4.7
4,0.0,0.0,0.947275,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,1.501853,4.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218,1.0,0.0,0.000000,0.722567,0.600270,0.074261,0.0,0.000000,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,2.701361,4.3
219,1.0,0.0,0.000000,0.665861,0.276581,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,3.749504,4.6
220,1.0,0.0,0.000000,0.516374,0.504677,0.093653,0.0,0.000000,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,2.484073,4.0
221,1.0,0.0,0.000000,0.448324,0.000000,0.230381,0.0,0.000000,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,3.113515,5.0


In [121]:
# Revisar el número de columnas en X_preprocessed
print("Número de columnas en X_preprocessed:", X_alto_preprocessed.shape[1])

Número de columnas en X_preprocessed: 68


# MODEL

In [ ]:
# pip install tensorflow

In [298]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np

def build_and_train_models(X_bajo, y_bajo, X_alto, y_alto):
    """
    Construye y entrena modelos neuronales para ambos grupos de reviews
    
    Args:
        X_bajo, y_bajo: Datos para productos con <60 reviews
        X_alto, y_alto: Datos para productos con >=60 reviews
        
    Returns:
        tuple: (model_bajo, model_alto, history_bajo, history_alto)
    """
    
    # Convertir matrices sparse a dense si es necesario
    X_bajo = X_bajo.toarray() if hasattr(X_bajo, "toarray") else X_bajo
    X_alto = X_alto.toarray() if hasattr(X_alto, "toarray") else X_alto
    
    # Arquitectura del modelo
    def build_model(input_shape):
        inputs = Input(shape=(input_shape,))
        x = Dense(128, activation='relu')(inputs)
        x = Dropout(0.3)(x)
        x = Dense(64, activation='relu')(x)
        outputs = Dense(1, activation='linear')(x)
        
        model = Model(inputs=inputs, outputs=outputs)
        model.compile(
            optimizer=Adam(learning_rate=0.001),
            loss='mse',
            metrics=['mae']
        )
        return model
    
    # Modelo para <60 reviews
    print("Entrenando modelo para <60 reviews...")
    model_bajo = build_model(X_bajo.shape[1])
    history_bajo = model_bajo.fit(
        X_bajo, y_bajo,
        epochs=100,
        batch_size=32,
        validation_split=0.2,
        callbacks=[EarlyStopping(patience=5, restore_best_weights=True)],
        verbose=1
    )
    
    # Modelo para >=60 reviews
    print("\nEntrenando modelo para >=60 reviews...")
    model_alto = build_model(X_alto.shape[1])
    history_alto = model_alto.fit(
        X_alto, y_alto,
        epochs=100,
        batch_size=32,
        validation_split=0.2,
        callbacks=[EarlyStopping(patience=5, restore_best_weights=True)],
        verbose=1
    )
    
    return model_bajo, model_alto, history_bajo, history_alto

# Uso con los datos de tu función Featuring
X_bajo, y_bajo, X_alto, y_alto, preprocessor = Featuring(df)
model_bajo, model_alto, hist_bajo, hist_alto = build_and_train_models(X_bajo, y_bajo, X_alto, y_alto)

Entrenando modelo para <60 reviews...
Epoch 1/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - loss: 18.2527 - mae: 4.1799 - val_loss: 15.0879 - val_mae: 3.8631
Epoch 2/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 12.3389 - mae: 3.4197 - val_loss: 9.5525 - val_mae: 3.0623
Epoch 3/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 6.7797 - mae: 2.4589 - val_loss: 4.8046 - val_mae: 2.1431
Epoch 4/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 3.4165 - mae: 1.6971 - val_loss: 1.4894 - val_mae: 1.1108
Epoch 5/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 1.4534 - mae: 1.0376 - val_loss: 0.3877 - val_mae: 0.5294
Epoch 6/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 1.0471 - mae: 0.7736 - val_loss: 0.5777 - val_mae: 0.5760
Epoch 7/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 1.4018 - mae: 0.9040 - val_loss: 0.4849 - val_mae: 0.5312
Epoch 8/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.9438 - mae: 0.7074 - val_loss: 0.3637 - val_mae: 0.5063
Epoch 9/100
6/6 ━━━━━━━━━━━━━━━

In [272]:
def predict_rating_auto(product_data, preprocessor, model_bajo, model_alto):
    """
    Predice el rating seleccionando automáticamente el modelo según las reviews
    
    Args:
        product_data: DataFrame con columnas ['Tipo', 'Product_Description', 'Price', 'Reviews']
        preprocessor: ColumnTransformer ajustado
        model_bajo: Modelo para <60 reviews
        model_alto: Modelo para >=60 reviews
        
    Returns:
        float: Rating predicho
    """
    # Extraer reviews y features
    reviews = product_data['Reviews'].values[0]
    features = product_data[['Tipo', 'Product_Description', 'Price']]
    
    # Preprocesamiento
    X_new = preprocessor.transform(features)
    X_new = X_new.toarray() if hasattr(X_new, "toarray") else X_new
    
    # Selección automática del modelo
    if reviews < 60:
        return model_bajo.predict(X_new, verbose=0)[0][0]
    else:
        return model_alto.predict(X_new, verbose=0)[0][0]

In [322]:
# Datos de ejemplo
nuevo_producto = pd.DataFrame({
    'Tipo': ['jabon'],
    'Product_Description': ['hecho con cafe y manteca de karite , cabello ,anticaida , vegetal, ecologico, vitamina c '],
    'Price': [12.99],
    'Reviews': [40]  # Seleccionará automáticamente el modelo_bajo
})

# Predicción
rating_pred = predict_rating_auto(
    nuevo_producto, 
    preprocessor,
    model_bajo,
    model_alto
)
print(f"Rating predicho: {rating_pred:.2f}")

Rating predicho: 4.08


In [291]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np

def evaluate_model(model, X, y_true):
    """
    Evalúa el modelo y retorna métricas clave
    
    Args:
        model: Modelo de Keras ya entrenado
        X: Features preprocesados
        y_true: Valores reales del target
        
    Returns:
        dict: Diccionario con las métricas calculadas
    """
    # Convertir a dense si es necesario
    X = X.toarray() if hasattr(X, "toarray") else X
    
    # Predicciones
    y_pred = model.predict(X, verbose=0).flatten()
    
    # Cálculo de métricas
    metrics = {
        'MAE': mean_absolute_error(y_true, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_true, y_pred)),
        'R2': r2_score(y_true, y_pred),
        'MSE': mean_squared_error(y_true, y_pred)
    }
    
    return metrics

In [293]:
# Obtener datos preprocesados (de tu función Featuring)
X_bajo, y_bajo, X_alto, y_alto, preprocessor = Featuring(df)

# Evaluar modelo para <60 reviews
metrics_bajo = evaluate_model(model_bajo, X_bajo, y_bajo)

# Evaluar modelo para >=60 reviews
metrics_alto = evaluate_model(model_alto, X_alto, y_alto)

# Mostrar resultados
print("=== Métricas para <60 reviews ===")
for k, v in metrics_bajo.items():
    print(f"{k}: {v:.4f}")

print("\n=== Métricas para >=60 reviews ===")
for k, v in metrics_alto.items():
    print(f"{k}: {v:.4f}")

=== Métricas para <60 reviews ===
MAE: 0.6358
RMSE: 0.8216
R2: -0.1250
MSE: 0.6750

=== Métricas para >=60 reviews ===
MAE: 0.7025
RMSE: 0.8570
R2: -9.6287
MSE: 0.7344


In [245]:
# import dtale
# from dtale.views import startup
# import dtale.app as dtale_app
# from pycaret.regression import *

In [243]:
# s=setup(data_bajo, target="Star_Rating", session_id=123)

In [241]:
# models()

In [239]:
# best = compare_models()

In [280]:
# Revisa estadísticas básicas
print(df[['Star_Rating', 'Reviews', 'Price']].describe())

# Verifica correlaciones
print(df[['Star_Rating', 'Reviews', 'Price']].corr())

# Ejemplo de salida esperada:
#               Star_Rating   Reviews     Price
# Star_Rating     1.000000  0.120458  0.034822
# Reviews         0.120458  1.000000  0.215873
# Price           0.034822  0.215873  1.000000

       Star_Rating     Reviews       Price
count   395.000000  395.000000  395.000000
mean      4.180506  150.805063   15.908456
std       0.612724  224.339029    9.393071
min       1.000000    1.000000    2.990000
25%       4.000000    5.000000    9.990000
50%       4.300000   34.000000   13.950000
75%       4.500000  206.500000   19.990000
max       5.000000  976.000000   70.520000
             Star_Rating   Reviews     Price
Star_Rating     1.000000  0.133793  0.061477
Reviews         0.133793  1.000000 -0.010761
Price           0.061477 -0.010761  1.000000


In [ ]:
# Definir tus palabras clave y compuestos
palabras_dominio = ['cabello','piel','facial','natural','tinte','pelo','corporal','barba',
                   'crecimiento','cuerpo','caida','parabenos','anticaida','vegano','canas','cara','fragancia',
                   'organico','afeitado','aroma','coloracion','delicada','ecologico','exfoliante','acne',
                   'fortalecer','graso','hidrata','poros','tradicionales','marina']

ingredientes_dominio = ['acido-salicilico','almendras','aloe-vera','aceites','carbon','antioxidante','arcilla','argan',
                       'cafe','cafeina','citric','coco','curcuma','esencial','ginseng','glicerina','jengibre','jojoba',
                       'karite','manteca-de-karite','menta','minerales','oliva','parfum','ricino','romero','rosa','sal','sandalo','zinc',
                       'canela','vitamina-c','madera','mango']

# Combinar vocabulario
vocabulario = palabras_dominio + ingredientes_dominio

# Crear el vectorizador con tu vocabulario
tfidf = TfidfVectorizer(
    tokenizer=procesar_texto,
    vocabulary=vocabulario,
    max_features=None
)

# Ajustar y transformar
X_tfidf = tfidf.fit_transform(df['Product_Description'])

# Ver las palabras clave seleccionadas
feature_names = tfidf.get_feature_names_out()


def clean_text(texto):
    if not isinstance(texto, str):
        return ""
    
    # Lista de términos compuestos (corregida)
    terminos_compuestos = [
        "acido salicilico",
        "acido hialuronico",
        "vitamina c",
        "aloe vera",
        "manteca de karite",
        "aceite almendras",
        "aceite argan",
        "aceite coco",
        "aceite ricino",
        "aceite jojoba",
        "aceite romero",
        "crecimiento capilar",
        "caida cabello"
    ]
    
    # Unir términos compuestos con guiones
    for termino in terminos_compuestos:
        texto = re.sub(
            r'\b' + re.escape(termino) + r'\b', 
            termino.replace(" ", "-"), 
            texto, 
            flags=re.IGNORECASE
        )
    
    # Limpieza estándar (conserva guiones)
    texto = re.sub(r'https?://\S+|www\.\S+', '', texto)  # URLs
    texto = unicodedata.normalize("NFD", texto).encode("ascii", "ignore").decode("utf-8")  # Tildes
    texto = re.sub(r'@\w+|#\w+', '', texto)  # Menciones/hashtags
    texto = re.sub(r'[^\w\s-]', '', texto)  # Conserva guiones
    texto = re.sub(r'\b\d+\b', '', texto)  # Números sueltos
    texto = texto.lower().strip()  # Minúsculas y espacios
    
    return texto


    def procesar_texto(texto):
    # Limpiar el texto
    texto_limpio = clean_text(texto)
    
    # Tokenizar (conserva guiones)
    tokens = texto_limpio.split()  # Split por espacios en lugar de word_tokenize
    
    # Filtrar stopwords en español
    stop_words = set(stopwords.words("spanish"))
    tokens = [token for token in tokens if token not in stop_words]
    

    # # Aplicar stemming (reducción a raíz)
    # stemmer = SnowballStemmer("spanish")
    # tokens = [stemmer.stem(token) for token in tokens]
    
    return tokens
def Featuring(df):
    copy = df.copy()
    
    #En lugar de crear una variable ponderada entre Star-Rating y Reviews, vamos a separar en categorias la variable Star_Rating y aplicar SMOTE para balancear la carga
    # bins = [0, 4, 4.4, 5.0]
    # labels = ['Baja', 'Media', 'Alta']
    # df['rating_category'] = pd.cut(df['Star_Rating'], bins=bins, labels=labels)

    # Verificar que las columnas necesarias existen
    required_columns = ['Tipo', 'Product_Description', 'Price', 'Reviews', 'Star_Rating']
    if not all(col in copy.columns for col in required_columns):
        raise ValueError(f"El DataFrame debe contener las siguientes columnas: {required_columns}")
    
    # Separar el dataset en dos grupos
    df_menos_60 = df[df['Reviews'] < 60]   # Datos con menos de 60 reviews
    df_mas_60 = df[df['Reviews'] >= 60] 

    # Definir transformaciones
    preprocessor = ColumnTransformer(
        transformers=[
            ('tipo', OneHotEncoder(drop='first'), ['Tipo']),  # Codificar 'Tipo'
            ('descripcion', TfidfVectorizer(
                tokenizer=procesar_texto,  # Usa tu función de tokenización
                vocabulary=vocabulario,    # ¡Tu vocabulario personalizado!
                max_features=None          # Ignora max_features si usas vocabulary
            ), 'Product_Description'),
            ('numericas', FunctionTransformer(np.log1p, validate=False), ['Price'])  
        ],
        remainder='drop'
    )
    
    # Aplicar preprocesamiento a cada subconjunto
    X_bajo = df_menos_60[['Tipo', 'Product_Description', 'Price']]
    y_bajo = df_menos_60['Star_Rating']

    X_alto = df_mas_60[['Tipo', 'Product_Description', 'Price']]
    y_alto = df_mas_60['Star_Rating']

    # Preprocesar características
    X_bajo_preprocessed = preprocessor.fit_transform(X_bajo)
    X_alto_preprocessed = preprocessor.transform(X_alto)  # Reutiliza el mismo preprocesador
    
    return X_bajo_preprocessed, y_bajo, X_alto_preprocessed, y_alto, preprocessor  # Retorna también el preprocesador para usarlo en Streamlit

    # Concatenar X_preprocessed y y, asegurando que los índices coincidan
data_bajo = pd.concat([pd.DataFrame(X_bajo_preprocessed.toarray()), y_bajo.reset_index(drop=True)], axis=1)
data_alto = pd.concat([pd.DataFrame(X_alto_preprocessed.toarray()), y_alto.reset_index(drop=True)], axis=1)